# 3. Optimization

1장에서 2개의 이미지를 비교해 특징점을 산출하고 유사한 특징을 가진 점끼리 매칭하였다. 2장에서는 특징점을 선별하여 4쌍의 매칭점 set을 정한 후 DLT solution을 적용하여 호모그래피를 산출했다. 앞서 산출한 수백개의 매칭점에 대해 각각 호모그래피를 계산할 수 있으며 이렇게 산출한 수 개의 호모그래피 중 한 개의 호모그래피를 특정해야 한다. 최소제곱법을 사용하여 간단하게 호모그래피를 추정할 수 있다.
  
그러나, 이때 잘못된 매칭점을 선별하게 되면 실제와는 완전히 다른 호모그래피가 산출되기 때문에 매칭점 선별 작업은 해당 과제의 신뢰성과 직결된다. 따라서 M-추정과 최소제곱중앙값, RANSAC 알고리즘 등을 이용해 잘못된 매칭점(아웃라이어)의 영향을 최소화하기 위한 작업이 필요하다.

## 3.1. 최소제곱법 (LSM, Least Square Method)
---

In [ ]:
def LSM(self,n_list,pts1,pts2):
    H = self.cal_homography(n_list,pts1,pts2)
    return H

In [ ]:
# LSM
H_lsm = find_homography.LSM(list(range(200)),pts1,pts2)

## 3.2. RANSAC 알고리즘
---
RANSAC(RANdom SAmple Consensus) 알고리즘은 컴퓨터 비전 분야에서 다양하게 사용되는 아웃라이어 제거를 위한 Model fitting 기법 중 하나이다. 주어진 data set에서 임의의 점들을 지정하여 해를 산출하고, 산출 해와 나머지 data set 점들의 오차값을 구하여 지정한 임계값보다 작은 오차를 가지는 점(인라이어)의 개수를 저장한다. 앞의 과정을 지정된 반복 횟수만큼 반복하며, 인라이어의 개수가 최대가 되는 해로 결과값을 업데이트 한다. 최적의 호모그래피를 구하기 위한 RANSAC 알고리즘은 다음과 같다.  

> 1. 오차 임계값과 반복 횟수 지정
> 2. 임의의 점 4개를 선택하여 호모그래피 산출
> 3. 앞서 산출한 호모그래피로 1번 이미지의 특징점을 변환
> 4. 3번에서 변환한 값과 2번 이미지의 특징점의 거리 계산 (오차 계산)
> 5. 오차 임계값보다 오차가 작은 점의 개수 저장 (인라이어 계산)
> 6. 기존에 저장된 인라이어 수와 비교하여 최대값을 가질 경우, 현재 호모그래피로 업데이트
<br> (현재 인라이어 수가 기존에 저장된 인라이어 수보다 작을 경우, 기존의 호모그래피 유지)
> 8. 지정한 횟수 만큼 위의 순서 반복

In [ ]:
def RANSAC(self,n_list,pts1,pts2,iter=500,e_limit=100):
    inlier_save = []
    H_save = []
    for i in range(iter):
      index_list = random.sample(n_list,4)
      H = self.cal_homography(index_list,pts1,pts2)

      # find inlier points
      inlier = []
      for p in n_list:
        p1 = np.append(pts1[p], np.array([1]))
        p2 = H@p1
        p2 = p2/p2[2]
        error = np.linalg.norm(np.append(pts2[p], np.array([1])) - p2)
        if error < e_limit:
          inlier.append(p)

      # update with homography with maximum number of inliers
      if H_save == []:
        H_save = H
      if inlier_save == 0:
        inlier_index_save = inlier
      if len(inlier) > len(inlier_save):
        H_save = H
        inlier_save = inlier

    return H_save, inlier_save

In [ ]:
# RANSAC
H_ran, _ = find_homography.RANSAC(list(range(200)),pts1,pts2,300,100)

## 3.3. LSM을 이용한 RANSAC 결과값 보정
---
RANSAC에서 구한 인라이어 점들로 최소제곱법을 진행하여 보다 정확한 값을 구할 수 있다.


In [ ]:
# RANSAC + LSM
H_temp,inlier = find_homography.RANSAC(list(range(200)),pts1,pts2,300,100)
H_ran_lsm = find_homography.LSM(inlier,pts1,pts2)

## 3.4. 결과 비교
---

In [ ]:
 H using OpenCV:
     [[ 3.41460497e+00 -9.48330647e-01  2.63175331e+02]
     [-6.83520031e-01  1.10122103e+00  9.12458003e+02]
     [-1.79370135e-04 -1.28011884e-03  1.00000000e+00]]

 H using LSM:
     [[-1.82696552e+00 -2.66199126e+00  1.01583795e+03]
     [-3.47362931e+00 -3.63516721e+00  1.59239727e+03]
     [-1.90948667e-03 -2.36320991e-03  1.00000000e+00]]

 H using RANSAC:
     [[ 4.38935323e+00 -9.31974201e-01  2.08187193e+02]
     [-2.17919794e-03  1.66839994e+00  8.37894668e+02]
     [ 2.00996563e-04 -1.12087718e-03  1.00000000e+00]]

 H using RANSAC + LSM:
     [[ 3.64411954e+00 -1.14568119e+00  2.90681048e+02]
     [-3.88742584e-01  1.04723628e+00  8.86663187e+02]
     [-3.16967985e-05 -1.34418032e-03  1.00000000e+00]]

OpenCV의 findHomography 함수와 비교하여   
> RANSAC + LSM  >  RANSAC  >  LSM  

순서로 유사한 값을 가지는 것을 확인할 수 있다.

---

작성자: 김윤중  
e-mail: yunjung.kim181@gmail.com  
github: YJ_Kim (@Muun-Muun)